# Jane Street: XGBoost HyperOpt + GroupKFold

I've made a notebook for hyperparameter tuning for XGBoost/LightGBM/CatBoost models. I use the Purged Rolling Time Series CV Split and GroupTimesSeriesSplit methods. I also try the forward-fillna method.

**References:**

https://www.kaggle.com/hamditarek/market-prediction-xgboost-with-gpu-fit-in-1min

https://www.kaggle.com/xhlulu/jane-street-cudf-xgboost-with-gpu

https://www.kaggle.com/jorijnsmit/found-the-holy-grail-grouptimeseriessplit

https://www.kaggle.com/marketneutral/purged-rolling-time-series-cv-split

https://www.kaggle.com/jorijnsmit/validating-cross-validators

1) Data memory reduction

2) Optuna Hyperoptimization

3) Weight as feature

4) ~Action def changed to: train['action'] =  (  (train['resp_1'] > 0 ) & (train['resp_2'] > 0 ) & (train['resp_3'] > 0 ) & (train['resp_4'] > 0 ) &  (train['resp'] > 0 )   ).astype('int')~

5) Repaired utility function

6) ~New features~ 

7) Action def changed to: train['action'] =  (  (train['resp'] > 0.0002 ) & (train['weight'] < 130)).astype('int')

8) New Optuna Hyperoptimization using utility score

9) AUTOENCODER FEATURES

10) ~NEW NEW FEATURES~

11) ~train['date'] > 85~

12) New ENCODER with NEW FEATURES

In [1]:
import os, gc
import numpy as np
from numba import njit
import datatable as dtable
import pandas as pd
import janestreet
import xgboost as xgb
from hyperopt import hp, fmin, tpe, Trials
from hyperopt.pyll.base import scope
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
from tqdm.notebook import tqdm
from joblib import dump, load
import pickle
import optuna

from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam, Adagrad
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf

#Limit Tensorflow GPU Usage (verry important)
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
# weighted average as per Donate et al.'s formula
# https://doi.org/10.1016/j.neucom.2012.02.053
def weighted_average(a):
    w = []
    n = len(a)
    for j in range(1, n + 1):
        j = 2 if j == 1 else j
        w.append(1 / (2**(n + 1 - j)))
    return np.average(a, weights = w)

# Preprocessing

In [3]:
n_splits = 5
group_gap = 31
add_encoder = True

In [4]:
%%time

print('Loading...')
#train = dtable.fread('/kaggle/input/jane-street-market-prediction/train.csv').to_pandas()
train = pickle.load(open('../input/janestreet-data-pickling/train.csv.pandas.pickle', 'rb'))
#NEW_FEATURES = pd.read_csv('../input/jane-street-feature-engineering/NEW_FEATURES.csv').set_index('ts_id')
features = [c for c in train.columns if 'feature' in c] + ['weight'] #+ NEW_FEATURES.columns.tolist()

print('Filling...')
train = train.query('weight > 0').reset_index(drop = True)

#train = pd.merge(train, NEW_FEATURES, how='left', left_on = ['ts_id'], right_index=True)

train[features] = train[features].fillna(method = 'ffill').fillna(0)

#train['action'] = (train['resp'] > 0).astype('int')
#train['action'] =  (  (train['resp_1'] > 0 ) & (train['resp_2'] > 0 ) & (train['resp_3'] > 0 ) & (train['resp_4'] > 0 ) &  (train['resp'] > 0 )   ).astype('int')
train['action'] =  (  (train['resp'] > 0.0002 ) & (train['weight'] < 130)).astype('int')

print('Finish.')

#train['feature_stock_id_sum'] = train['feature_41'] + train['feature_42'] + train['feature_43']
#train['feature_1_2_cross'] = train['feature_1']/(train['feature_2']+1e-5)
#train['feature_34_43_cross'] = ( (train['feature_34'] > 0) & (train['feature_43'] > 0) ).astype('int')

#features = features + ['feature_stock_id_sum', 'feature_1_2_cross', 'feature_34_43_cross']

Loading...
Filling...
Finish.
CPU times: user 3.05 s, sys: 2.97 s, total: 6.02 s
Wall time: 18.2 s


# PurgedTimeSeriesKFold & GroupTimeSplitKFold

In [5]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

# Add Encoder Features

In [6]:
# From https://medium.com/@micwurm/using-tensorflow-lite-to-speed-up-predictions-a3954886eb98

class LiteModel:
    
    @classmethod
    def from_file(cls, model_path):
        return LiteModel(tf.lite.Interpreter(model_path=model_path))
    
    @classmethod
    def from_keras_model(cls, kmodel):
        converter = tf.lite.TFLiteConverter.from_keras_model(kmodel)
        tflite_model = converter.convert()
        return LiteModel(tf.lite.Interpreter(model_content=tflite_model))
    
    def __init__(self, interpreter):
        self.interpreter = interpreter
        self.interpreter.allocate_tensors()
        input_det = self.interpreter.get_input_details()[0]
        output_det = self.interpreter.get_output_details()[0]
        self.input_index = input_det["index"]
        self.output_index = output_det["index"]
        self.input_shape = input_det["shape"]
        self.output_shape = output_det["shape"]
        self.input_dtype = input_det["dtype"]
        self.output_dtype = output_det["dtype"]
        
    def predict(self, inp):
        inp = inp.astype(self.input_dtype)
        count = inp.shape[0]
        out = np.zeros((count, self.output_shape[1]), dtype=self.output_dtype)
        for i in range(count):
            self.interpreter.set_tensor(self.input_index, inp[i:i+1])
            self.interpreter.invoke()
            out[i] = self.interpreter.get_tensor(self.output_index)[0]
        return out
    
    def predict_single(self, inp):
        """ Like predict(), but only for a single record. The input data can be a Python list. """
        inp = np.array([inp], dtype=self.input_dtype)
        self.interpreter.set_tensor(self.input_index, inp)
        self.interpreter.invoke()
        out = self.interpreter.get_tensor(self.output_index)
        return out[0]

In [7]:
def create_encoder(input_dim,output_dim,noise=0.05):
    i = Input(input_dim)
    encoded = BatchNormalization()(i)
    #encoded = GaussianNoise(noise)(encoded)
    encoded = Dense(128,activation='relu')(encoded)

    encoded = BatchNormalization()(encoded)
    encoded = Dropout(0.2)(encoded)
    encoded = Dense(64,activation='relu')(encoded)
    
    decoded = BatchNormalization()(encoded)
    decoded = Dropout(0.2)(decoded)
    decoded = Dense(128,activation='relu')(decoded)
    decoded = BatchNormalization()(decoded)
    decoded = Dropout(0.2)(decoded)
    decoded = Dense(input_dim,name='decoded')(decoded)
    
    x = Dense(32)(decoded)
    x = BatchNormalization()(x)
    x = Lambda(tf.keras.activations.swish)(x)
    x = Dropout(0.2)(x)
    x = Dense(16)(x)
    x = BatchNormalization()(x)
    x = Lambda(tf.keras.activations.swish)(x)
    x = Dropout(0.2)(x)
    x = Dense(output_dim,activation='sigmoid',name='label_output')(x)
    
    encoder = Model(inputs=i,outputs=encoded)
    decoder = Model(inputs=i,outputs=decoded)
    autoencoder = Model(inputs=i,outputs=[decoded,x])
    
    autoencoder.compile(optimizer=Adam(0.01),loss={'decoded':'mse','label_output':'binary_crossentropy'})
    return autoencoder, decoder, encoder

In [8]:
%%time
if add_encoder:
    autoencoder, decoder, encoder = create_encoder(train[features].shape[-1],1,noise=0)
    encoder.trainable = False

    encoder.load_weights(f'../input/jane-street-bottleneck-encoder-fe-train/encoder_62.tf')
    encoder = LiteModel.from_keras_model(encoder)


    enc_fe = encoder.predict(train[features].values)
    enc_fe = pd.DataFrame(enc_fe)
    enc_fe.columns = ['enc_fe' + str(col_name)  for col_name in enc_fe.columns]
    
    train = pd.concat([train,enc_fe], axis=1)

CPU times: user 31.8 s, sys: 2.24 s, total: 34 s
Wall time: 36.2 s


In [9]:
#Update Features
#features = [c for c in train.columns if 'feature' in c] + ['weight'] + [c for c in enc_fe.columns if 'enc' in c] #+ NEW_FEATURES.columns.tolist()
#After Feature Selection
#features = ['feature_43', 'feature_44', 'feature_41', 'feature_45', 'feature_42', 'feature_5', 'feature_6', 'feature_62', 'feature_83', 'feature_60', 'feature_3', 'feature_63', 'feature_4', 'feature_39', 'feature_40', 'feature_61', 'feature_38', 'feature_77', 'feature_37', 'feature_64', 'feature_121', 'feature_120', 'feature_55', 'feature_107', 'feature_125', 'feature_95', 'feature_101', 'feature_89', 'feature_124', 'feature_90', 'feature_65', 'feature_67', 'feature_57', 'feature_102', 'feature_119', 'enc_fe20', 'feature_84', 'feature_113', 'enc_fe26', 'feature_66', 'feature_127', 'feature_78', 'feature_27', 'enc_fe18', 'enc_fe16', 'feature_7', 'enc_fe63', 'feature_8', 'feature_28', 'enc_fe23', 'feature_96', 'enc_fe29', 'feature_58', 'enc_fe28', 'enc_fe39', 'feature_18', 'enc_fe33', 'enc_fe62', 'enc_fe7', 'enc_fe44', 'enc_fe10', 'enc_fe6', 'enc_fe36', 'enc_fe12', 'enc_fe19', 'enc_fe14', 'feature_17', 'feature_68', 'feature_114', 'enc_fe8', 'enc_fe21', 'enc_fe38', 'enc_fe57', 'enc_fe27', 'enc_fe32', 'feature_53', 'enc_fe54', 'feature_126', 'enc_fe53', 'enc_fe46', 'feature_56', 'feature_108', 'enc_fe56', 'enc_fe2', 'feature_72', 'enc_fe59', 'feature_59', 'enc_fe1', 'feature_92', 'enc_fe30', 'enc_fe25', 'enc_fe55', 'enc_fe61', 'feature_70', 'enc_fe9', 'enc_fe40', 'feature_71', 'enc_fe50', 'enc_fe48', 'enc_fe49', 'enc_fe52', 'enc_fe3', 'enc_fe17', 'enc_fe58', 'feature_49', 'feature_54', 'enc_fe35', 'enc_fe31', 'enc_fe24', 'feature_69', 'enc_fe0', 'feature_129', 'feature_110', 'enc_fe37', 'enc_fe22', 'feature_86', 'enc_fe4', 'feature_123', 'enc_fe15', 'feature_20', 'enc_fe43', 'enc_fe42', 'enc_fe13', 'feature_104', 'feature_116', 'feature_98', 'feature_30', 'enc_fe5', 'enc_fe51', 'feature_11', 'enc_fe41', 'feature_128', 'feature_50', 'feature_117', 'feature_80', 'feature_12', 'enc_fe47', 'feature_52', 'feature_31', 'feature_32', 'feature_122', 'feature_111', 'feature_21', 'weight', 'feature_46', 'feature_105', 'feature_99', 'feature_2', 'feature_33', 'feature_19', 'feature_74', 'feature_48', 'feature_47', 'enc_fe11', 'feature_22', 'feature_93', 'feature_10', 'feature_51', 'feature_34', 'enc_fe45', 'feature_109', 'feature_16', 'feature_1', 'feature_87', 'feature_23', 'feature_0']
#features = features + ['feature_stock_id_sum', 'feature_1_2_cross', 'feature_34_43_cross'] 

features_enc = [c for c in enc_fe.columns if 'enc' in c]
train = train[features_enc + ['action', 'date', 'weight', 'resp']]

# Training

In [10]:
%%time

p_best =   {'n_estimators': 957,
            'max_depth': 11,
            'learning_rate': 0.011135865455082924,
            'subsample': 0.7393122937861645,
            'colsample_bytree': 0.8702159472037736,
            'gamma': 9,
            'verbosity': 0, 
            'objective': 'binary:logistic',
            'eval_metric': 'auc', 
            'tree_method': 'gpu_hist', 
            'random_state': 42,   
            }
    
#X_tr, y_tr = train.loc[train['date'] > 85, features_enc].values, train.loc[train['date'] > 85, 'action'].values
X_tr, y_tr = train[features_enc].values, train['action'].values
d_tr = xgb.DMatrix(X_tr, y_tr)

# Seed Blending
models = []
for seed in range(5):
    p_best['random_state'] = seed
    clf = xgb.train(p_best, d_tr, p_best['n_estimators'])
    
    tr_pred = clf.predict(d_tr)
    score = roc_auc_score(y_tr, tr_pred)
    print(f'ROC AUC:\t', score)
    
    models.append(clf)
    
    rubbish = gc.collect()

ROC AUC:	 0.6424502254289515
ROC AUC:	 0.6427413807120198
ROC AUC:	 0.6439523122624801
ROC AUC:	 0.6423259932542875
ROC AUC:	 0.6426502481255298
CPU times: user 10min 35s, sys: 1.99 s, total: 10min 37s
Wall time: 7min 26s


# Submitting

In [11]:
@njit
def fast_fillna(array, values):
    if np.isnan(array.sum()):
        array = np.where(np.isnan(array), values, array)
    return array

#train.loc[0, features[1:]] = fast_fillna(train.loc[0, features[1:]].values, 0)

In [12]:
env = janestreet.make_env()
env_iter = env.iter_test()

In [13]:
opt_th = 0.50125
tmp = np.zeros(len(features))
for (test_df, pred_df) in tqdm(env_iter):
    if test_df['weight'].item() > 0:
        x_tt = test_df.loc[:, features].values
        x_tt[0, :] = fast_fillna(x_tt[0, :], tmp)
        tmp = x_tt[0, :]
        x_tt2 = encoder.predict(x_tt)
        d_tt2 = xgb.DMatrix(x_tt2)
        pred = 0.
        for clf in models:
            pred += clf.predict(d_tt2) / len(models)
        pred_df.action = np.where(pred >= opt_th, 1, 0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)

|          | 0/? [00:00<?, ?it/s]